In [1]:
data="https://raw.githubusercontent.com/alexeygrigorev/datasets/master/car_fuel_efficiency.csv"
!wget -O fuel.csv $data

--2026-01-16 11:33:10--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/car_fuel_efficiency.csv
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 874188 (854K) [text/plain]
Saving to: ‘fuel.csv’

fuel.csv            100%[===================>] 853.70K  --.-KB/s    in 0.05s   

2026-01-16 11:33:10 (16.1 MB/s) - ‘fuel.csv’ saved [874188/874188]



In [3]:
import pandas as pd
df = pd.read_csv('fuel.csv')
df.head()

,engine_displacement,num_cylinders,horsepower,vehicle_weight,acceleration,model_year,origin,fuel_type,drivetrain,num_doors,fuel_efficiency_mpg
0,170,3.0,159.0,3413.433759,17.7,2003,Europe,Gasoline,All-wheel drive,0.0,13.231729
1,130,5.0,97.0,3149.664934,17.8,2007,USA,Gasoline,Front-wheel drive,0.0,13.688217
2,170,NaN,78.0,3079.038997,15.1,2018,Europe,Gasoline,Front-wheel drive,0.0,14.246341
3,220,4.0,NaN,2542.392402,20.2,2009,USA,Diesel,All-wheel drive,2.0,16.912736
4,210,1.0,140.0,3460.870990,14.4,2009,Europe,Gasoline,All-wheel drive,2.0,12.488369


In [5]:
# Data prep

df.isnull().sum()

engine_displacement      0
num_cylinders          482
horsepower             708
vehicle_weight           0
acceleration           930
model_year               0
origin                   0
fuel_type                0
drivetrain               0
num_doors              502
fuel_efficiency_mpg      0
dtype: int64

In [8]:
from sklearn.model_selection import train_test_split
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_full_train.fillna(0, inplace=True)

df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

y_train = df_train.fuel_efficiency_mpg
y_val = df_val.fuel_efficiency_mpg
y_full_train = df_full_train.fuel_efficiency_mpg
y_test = df_test.fuel_efficiency_mpg

del df_full_train['fuel_efficiency_mpg']
del df_train['fuel_efficiency_mpg']
del df_val['fuel_efficiency_mpg']
del df_test['fuel_efficiency_mpg']

# 1

In [17]:
from sklearn.feature_extraction import DictVectorizer
dv = DictVectorizer(sparse=True)
train_dicts = df_train.to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

In [18]:

# we train a decition tree regressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_text
model = DecisionTreeRegressor(max_depth=1, min_samples_leaf=5, random_state=1)
model.fit(X_train, y_train)
print(export_text(model, feature_names=dv.get_feature_names_out().tolist()))

|--- vehicle_weight <= 3022.11
|   |--- value: [16.88]
|--- vehicle_weight >  3022.11
|   |--- value: [12.94]



# 2

In [24]:
# random forest regressor
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=10, random_state=1, n_jobs=-1)

X_val = dv.transform(df_val.to_dict(orient='records'))

model.fit(X_train, y_train)
y_pred = model.predict(X_val)

# rmse
from sklearn.metrics import root_mean_squared_error
import numpy as np
rmse = root_mean_squared_error(y_val, y_pred)
print(rmse)

0.45957772230927263


# 3

In [26]:
# try n_estimators from 10 to 200 with step 10
for n_estimators in range(10, 201, 10):
    model = RandomForestRegressor(n_estimators=n_estimators, random_state=1, n_jobs=-1)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    rmse = root_mean_squared_error(y_val, y_pred)
    # print with 3 decimal places
    print(f'n_estimators: {n_estimators}, rmse: {rmse:.3f}')

n_estimators: 10, rmse: 0.460
n_estimators: 20, rmse: 0.454
n_estimators: 30, rmse: 0.452
n_estimators: 40, rmse: 0.449
n_estimators: 50, rmse: 0.447
n_estimators: 60, rmse: 0.445
n_estimators: 70, rmse: 0.445
n_estimators: 80, rmse: 0.445
n_estimators: 90, rmse: 0.445
n_estimators: 100, rmse: 0.445
n_estimators: 110, rmse: 0.444
n_estimators: 120, rmse: 0.444
n_estimators: 130, rmse: 0.444
n_estimators: 140, rmse: 0.443
n_estimators: 150, rmse: 0.443
n_estimators: 160, rmse: 0.443
n_estimators: 170, rmse: 0.443
n_estimators: 180, rmse: 0.442
n_estimators: 190, rmse: 0.442
n_estimators: 200, rmse: 0.442


# 4

In [28]:
for max_depth in [10, 15, 20, 25]:
    # calculate mean rmse for n_estimators from 10 to 200 with step 10
    mean_rmse = 0
    for n_estimators in range(10, 201, 10):
        model = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, random_state=1, n_jobs=-1)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        rmse = root_mean_squared_error(y_val, y_pred)
        mean_rmse += rmse

    mean_rmse /= 20
    print(f'max_depth: {max_depth}, mean_rmse: {mean_rmse:.3f}')

max_depth: 10, mean_rmse: 0.442
max_depth: 15, mean_rmse: 0.445
max_depth: 20, mean_rmse: 0.446
max_depth: 25, mean_rmse: 0.446


# 5

In [29]:
# feature importance
# gain = reduction in impurity at each split

model = RandomForestRegressor(n_estimators=10, max_depth=20, random_state=1, n_jobs=-1)
model.fit(X_train, y_train)
for name, score in zip(dv.get_feature_names_out().tolist(), model.feature_importances_):
    print(f'{name}: {score}')

acceleration: 0.01147970063142938
drivetrain=All-wheel drive: 0.0003571085493021933
drivetrain=Front-wheel drive: 0.00034538411263183535
engine_displacement: 0.0032727919136094864
fuel_type=Diesel: 0.000325424322869738
fuel_type=Gasoline: 0.00036038360069172865
horsepower: 0.015997897714266237
model_year: 0.003212300094794675
num_cylinders: 0.0023433469524512048
num_doors: 0.0016349895439306998
origin=Asia: 0.0004622464955097423
origin=Europe: 0.000518739638586969
origin=USA: 0.0005397216891829147
vehicle_weight: 0.9591499647407432


# 6

In [32]:
# train with xgboost
import xgboost as xgb
d_train = xgb.DMatrix(X_train, label=y_train, feature_names=dv.get_feature_names_out().tolist())
d_val = xgb.DMatrix(X_val, label=y_val, feature_names=dv.get_feature_names_out().tolist())

params = {
    'eta': 0.3,
    'max_depth': 6,
    'min_child_weight': 1,

    'objective': 'reg:squarederror',
    'nthread': 8,

    'seed': 1,
    'verbosity': 1,
}

model = xgb.train(params, d_train, num_boost_round=100)

# evaluate rmse
y_pred = model.predict(d_val)
rmse = root_mean_squared_error(y_val, y_pred)
print(rmse)

0.45017755678087246


In [33]:
params = {
    'eta': 0.1,
    'max_depth': 6,
    'min_child_weight': 1,

    'objective': 'reg:squarederror',
    'nthread': 8,

    'seed': 1,
    'verbosity': 1,
}

model = xgb.train(params, d_train, num_boost_round=100)

# evaluate rmse
y_pred = model.predict(d_val)
rmse = root_mean_squared_error(y_val, y_pred)
print(rmse)

0.42622800553359225
